In [8]:
library(data.table)
library(tidyverse)
library(lme4)
library(performance)
library(cowplot)
library(emmeans)
library(marginaleffects)
library(pbapply)

In [9]:
dt <- fread("../../natera_recomb/analysis/co_post_process/results/v30b_heuristic_90_nsib_qual.crossover_filt.deCode_haldorsson19.crossover_count.maternal.euploid.csv.gz") %>%
  .[, is_aneuploid_embryo := FALSE]

dt2 <- fread("../../natera_recomb/analysis/co_post_process/results/v30b_heuristic_90_nsib_qual.crossover_filt.deCode_haldorsson19.crossover_count.maternal.aneuploid.csv.gz") %>%
  .[, is_aneuploid_embryo := TRUE]

dt <- rbind(dt, dt2) %>%
  .[bf_max_cat == "2"]

dt[, AVGSIGMA := weighted.mean(sigma_baf, cM_len), by = uid]
dt[, AVGPI0 := weighted.mean(pi0_baf, cM_len), by = uid]

dt[egg_donor == 1, patient_age := as.numeric(25)]
df <- dt %>% filter((egg_donor == 0) & (sperm_donor == 0))
head(df)

chrom,uid,IID,nco,patient_age,egg_donor,sperm_donor,avg_pi0,avg_sigma,maternal_meiotic_aneuploidy,⋯,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,is_aneuploid_embryo
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
chr16,10005770025_R06C01+10005770025_R05C01+3999947101_R01C01,10005770025_R06C01,1,43.2411,0,0,0.5239479,0.1781051,FALSE,⋯,0.000866399,0.00357098,-0.00764743,-0.00283709,-0.004159,0.00249129,0.00157069,-0.00210666,0.00270483,FALSE
chr14,10005770025_R06C01+10005770025_R05C01+3999947101_R01C01,10005770025_R06C01,3,43.2411,0,0,0.5643161,0.1481064,FALSE,⋯,0.000866399,0.00357098,-0.00764743,-0.00283709,-0.004159,0.00249129,0.00157069,-0.00210666,0.00270483,FALSE
chr22,10005770025_R06C01+10005770025_R05C01+3999947101_R01C01,10005770025_R06C01,1,43.2411,0,0,0.4764777,0.1941526,FALSE,⋯,0.000866399,0.00357098,-0.00764743,-0.00283709,-0.004159,0.00249129,0.00157069,-0.00210666,0.00270483,FALSE
chr9,10005770025_R06C01+10005770025_R05C01+3999947101_R01C01,10005770025_R06C01,1,43.2411,0,0,0.5952933,0.1581071,FALSE,⋯,0.000866399,0.00357098,-0.00764743,-0.00283709,-0.004159,0.00249129,0.00157069,-0.00210666,0.00270483,FALSE
chr2,10005770025_R06C01+10005770025_R05C01+3999947101_R01C01,10005770025_R06C01,2,43.2411,0,0,0.5877440,0.1399988,FALSE,⋯,0.000866399,0.00357098,-0.00764743,-0.00283709,-0.004159,0.00249129,0.00157069,-0.00210666,0.00270483,FALSE
chr8,10005770025_R06C01+10005770025_R05C01+3999947101_R01C01,10005770025_R06C01,2,43.2411,0,0,0.5958460,0.1407646,FALSE,⋯,0.000866399,0.00357098,-0.00764743,-0.00283709,-0.004159,0.00249129,0.00157069,-0.00210666,0.00270483,FALSE


In [10]:
# Just looking at the standard effect-direction 
df %>% group_by(maternal_meiotic_aneuploidy) %>% summarize(avg=mean(nco), n=n(), sd=sd(nco), se=sd/sqrt(n))

maternal_meiotic_aneuploidy,avg,n,sd,se
<lgl>,<dbl>,<int>,<dbl>,<dbl>
FALSE,2.228299,985240,1.700726,0.001713418
TRUE,2.198947,503847,1.758620,0.002477551


In [4]:
# fit model
m1 <- glmer(data = df, 
            formula = nco ~ (1 | IID / uid) + scale(patient_age) + PC1 + PC2 + PC3 + PC4 + PC5 + 
              PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 +
              PC18 + PC19 + PC20 + offset(log(cM_len)) + scale(AVGSIGMA) + scale(AVGPI0) + scale(NEMBRYO) + maternal_meiotic_aneuploidy,
            family = poisson,
            nAGQ = 0,
            control = glmerControl(optimizer = "bobyqa"))

summary(m1)


Correlation matrix not shown by default, as p = 26 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 0) [glmerMod]
 Family: poisson  ( log )
Formula: nco ~ (1 | IID/uid) + scale(patient_age) + PC1 + PC2 + PC3 +  
    PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 +  
    PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + offset(log(cM_len)) +  
    scale(AVGSIGMA) + scale(AVGPI0) + scale(NEMBRYO) + maternal_meiotic_aneuploidy
   Data: df
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
 5197247  5197589 -2598596  5197191  1489059 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5217 -0.7021 -0.1072  0.5232 17.1537 

Random effects:
 Groups  Name        Variance Std.Dev.
 uid:IID (Intercept) 0.01908  0.1381  
 IID     (Intercept) 0.02340  0.1530  
Number of obs: 1489087, groups:  uid:IID, 70538; IID, 13326

Fixed effects:
                                  Estimate Std. Error   z value Pr(>|z|)    
(Intercept)       

In [5]:
emmeans(m1, "maternal_meiotic_aneuploidy", type = "response")

 maternal_meiotic_aneuploidy  rate       SE  df asymp.LCL asymp.UCL
 FALSE                       2.217 0.003916 Inf     2.209     2.225
  TRUE                       2.112 0.004471 Inf     2.103     2.121

Confidence level used: 0.95 
Intervals are back-transformed from the log scale 

In [6]:
# > emmeans(m1, "is_aneuploid_embryo", type = "response")
#  is_aneuploid_embryo  rate       SE  df asymp.LCL asymp.UCL
#  FALSE               2.239 0.003686 Inf     2.231     2.246
#   TRUE               2.092 0.003920 Inf     2.085     2.100

In [7]:
summary(m1)$coefficients[,4]

(Intercept)              scale(patient_age) 
                   0.000000e+00                    9.875954e-01 
                            PC1                             PC2 
                  2.172062e-185                    7.077171e-65 
                            PC3                             PC4 
                   4.124271e-07                    1.064434e-58 
                            PC5                             PC6 
                   1.362714e-01                    9.339826e-01 
                            PC7                             PC8 
                   9.051232e-01                    4.061292e-33 
                            PC9                            PC10 
                   5.704562e-82                    6.128453e-04 
                           PC11                            PC12 
                   2.072727e-30                    9.582783e-04 
                           PC13                            PC14 
                   2.451475e-02                    4.336513e-01 
                           PC15                            PC16 
                   5.844833e-01                    2.072032e-01 
                           PC17                            PC18 
                   1.605550e-01                    5.974105e-02 
                           PC19                            PC20 
                   2.575125e-01                    3.366995e-02 
                scale(AVGSIGMA)                   scale(AVGPI0) 
                   1.814192e-14                    6.941507e-71 
                 scale(NEMBRYO) maternal_meiotic_aneuploidyTRUE 
                   1.263404e-10                   4.645305e-150